1. Imports & settings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (10,5)


2. Load dataset

In [ ]:
df = pd.read_csv("SriLanka_Weather_Dataset.csv")
print("Loaded rows:", df.shape[0])
df.head()

Loaded rows: 15990


,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,shortwave_radiation_sum,precipitation_sum,rain_sum,snowfall_sum,...,et0_fao_evapotranspiration,latitude,longitude,elevation,city,year,month,day,dayofyear,week
0,31.1,24.1,26.6,36.6,28.7,31.3,19.71,6.7,6.7,0,...,4.05,7.0,79.899994,16,Colombo,2022,1,1,1,52
1,31.3,24.2,26.9,37.7,28.2,31.3,19.53,2.1,2.1,0,...,4.19,7.0,79.899994,16,Colombo,2022,1,2,2,52
2,30.3,24.4,26.6,35.1,27.5,30.2,16.48,11.2,11.2,0,...,3.81,7.0,79.899994,16,Colombo,2022,1,3,3,1
3,30.3,24.5,26.7,34.4,27.1,29.8,17.21,3.9,3.9,0,...,4.12,7.0,79.899994,16,Colombo,2022,1,4,4,1
4,29.8,24.3,26.7,33.2,27.2,29.8,11.93,5.6,5.6,0,...,3.16,7.0,79.899994,16,Colombo,2022,1,5,5,1


3. Basic preprocessing

In [ ]:
if 'time' in df.columns:
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month
    df['day'] = df['time'].dt.day
    df['dayofyear'] = df['time'].dt.dayofyear
    df['week'] = df['time'].dt.isocalendar().week.astype(int)
    df = df.drop(columns=['time'])

df['city'] = df['city'].astype(str)

print("Columns:", df.columns.tolist())

Columns: ['temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'apparent_temperature_max', 'apparent_temperature_min', 'apparent_temperature_mean', 'shortwave_radiation_sum', 'precipitation_sum', 'rain_sum', 'snowfall_sum', 'precipitation_hours', 'windspeed_10m_max', 'windgusts_10m_max', 'winddirection_10m_dominant', 'et0_fao_evapotranspiration', 'latitude', 'longitude', 'elevation', 'city', 'year', 'month', 'day', 'dayofyear', 'week']


4. Graphical evaluation

In [ ]:
os.makedirs("figures", exist_ok=True)

4.1 bold text Bar chart: rows per
city (count)

In [ ]:
city_counts = df['city'].value_counts()
plt.figure()
city_counts.plot(kind='bar')
plt.title("Rows per city (data coverage)")
plt.ylabel("Count")
plt.xlabel("City")
plt.tight_layout()
plt.savefig("figures/city_counts.png")
plt.close()

4.2. Correlation matrix

In [ ]:
numeric = df.select_dtypes(include=[np.number])
corr = numeric.corr()
plt.figure(figsize=(12,10))
sns.heatmap(corr, annot=False, cmap='coolwarm', center=0)
plt.title("Correlation matrix (numeric features)")
plt.tight_layout()
plt.savefig("figures/correlation_matrix.png")
plt.close()

4.3. Distribution plots for target variables

In [ ]:
target_cols = ['rain_sum', 'windspeed_10m_max', 'windgusts_10m_max', 'precipitation_hours']
for col in target_cols:
    plt.figure()
    sns.histplot(df[col], bins=50, kde=True)
    plt.title(f"Distribution: {col}")
    plt.tight_layout()
    plt.savefig(f"figures/dist_{col}.png")
    plt.close()

4.4. Boxplots for targets

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=df[target_cols])
plt.title("Boxplots of target variables")
plt.tight_layout()
plt.savefig("figures/boxplots_targets.png")
plt.close()

print("Saved figure files to ./figures/")

Saved figure files to ./figures/


5. Feature engineering: lag features for next-day forecasting

In [ ]:
df_sorted = df.sort_values(['city','year','month','day']).reset_index(drop=True)

def create_lags(df, cols, lags=[1,2,3]):
    out = df.copy()
    for col in cols:
        for lag in lags:
            out[f"{col}_lag{lag}"] = out.groupby('city')[col].shift(lag)
    return out

lag_cols = ['rain_sum','windspeed_10m_max','windgusts_10m_max','precipitation_hours',
            'temperature_2m_mean','shortwave_radiation_sum']
df_lag = create_lags(df_sorted, lag_cols, lags=[1,2,3])

df_lag = df_lag.dropna().reset_index(drop=True)
print("After lagging rows:", df_lag.shape[0])

After lagging rows: 15900


6. Prepare X and Y

In [ ]:
TARGETS = ['rain_sum','windspeed_10m_max','windgusts_10m_max','precipitation_hours']

drop_features = TARGETS[:]
X = df_lag.drop(columns=drop_features)
Y = df_lag[TARGETS]

print("X shape:", X.shape, "Y shape:", Y.shape)

X shape: (15900, 38) Y shape: (15900, 4)


7. Encode city, scale numeric features

In [ ]:
le_city = LabelEncoder()
X['city'] = le_city.fit_transform(X['city'])

numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
numeric_features.remove('city')

scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[numeric_features] = scaler.fit_transform(X[numeric_features])


joblib.dump(scaler, "climatrix_scaler.pkl")
joblib.dump(le_city, "climatrix_city_encoder.pkl")

['climatrix_city_encoder.pkl']

8. Train/test split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_scaled, Y, test_size=0.2, random_state=42, shuffle=True
)
print("Train/test sizes:", X_train.shape, X_test.shape)

Train/test sizes: (12720, 38) (3180, 38)


9. Train 5 models and evaluate

In [ ]:
models = {
    "Linear Regression": MultiOutputRegressor(LinearRegression()),
    "Random Forest": MultiOutputRegressor(RandomForestRegressor(n_estimators=150, random_state=42)),
    "Gradient Boosting": MultiOutputRegressor(GradientBoostingRegressor(random_state=42)),
    "XGBoost": None,
    "KNN Regressor": MultiOutputRegressor(KNeighborsRegressor(n_neighbors=5))
}

try:
    from xgboost import XGBRegressor
    models["XGBoost"] = MultiOutputRegressor(XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=6, subsample=0.8, random_state=42))
except Exception as e:
    print("XGBoost not available:", e)
    models.pop("XGBoost", None)

results = []
trained_models = {}

for name, model in models.items():
    print("Training:", name)
    model.fit(X_train, Y_train)
    preds = model.predict(X_test)
    r2 = r2_score(Y_test, preds, multioutput='uniform_average')
    mse = mean_squared_error(Y_test, preds, multioutput='uniform_average')
    rmse = np.sqrt(mse)
    results.append([name, r2, mse, rmse])
    trained_models[name] = model

results_df = pd.DataFrame(results, columns=['Model','R2','MSE','RMSE']).sort_values(by='R2', ascending=False)
display(results_df)

Training: Linear Regression
Training: Random Forest
Training: Gradient Boosting
Training: XGBoost
Training: KNN Regressor


,Model,R2,MSE,RMSE
1,Random Forest,0.939867,4.207445,2.051206
3,XGBoost,0.939011,4.276822,2.068048
2,Gradient Boosting,0.895760,7.359882,2.712910
0,Linear Regression,0.863291,9.511465,3.084066
4,KNN Regressor,0.791187,14.373211,3.791202


10. Select best model and save

In [ ]:
best_name = results_df.iloc[0]['Model']
best_model = trained_models[best_name]
print("Selected best model:", best_name)

joblib.dump(best_model, "climatrix_best_model.pkl")
print("Saved model to climatrix_best_model.pkl")

Selected best model: Random Forest
Saved model to climatrix_best_model.pkl


11. Prediction utility

In [27]:
def classify_risk_from_preds(rain, wind, gust, precip_hours):

    if (rain > 50) or (gust > 60) or (wind > 25 and rain > 30):
        overall = "High"
    elif (rain > 20) or (gust > 40) or (wind > 15):
        overall = "Moderate"
    else:
        overall = "Low"
    return overall

In [28]:
def prepare_input_row(base_row):
    row = base_row.copy()

    if isinstance(row['city'], str):
        row['city'] = le_city.transform([row['city']])[0]

    row_scaled = row.copy()
    row_scaled[numeric_features] = scaler.transform(row_scaled[numeric_features].values.reshape(1,-1))[0]
    return row_scaled

In [29]:
def iterative_forecast(start_row, horizon=1):
    """
    start_row: pandas Series containing the features of the last available day for a city (unscaled)
    horizon: int days ahead (1 = next-day)
    Returns: DataFrame with horizon rows of predictions (rain, wind, gust, precip_hours)
    """
    current = start_row.copy()
    preds_list = []
    for h in range(horizon):

        cur_scaled = current.copy()

        if isinstance(cur_scaled['city'], str):
            cur_scaled['city'] = le_city.transform([cur_scaled['city']])[0]
        cur_scaled[numeric_features] = scaler.transform(cur_scaled[numeric_features].values.reshape(1,-1))[0]

        yhat = best_model.predict(cur_scaled.values.reshape(1,-1))[0]
        rain_pred, wind_pred, gust_pred, prec_hours_pred = yhat
        preds_list.append({
            'day': h+1,
            'rain_sum': float(rain_pred),
            'windspeed_10m_max': float(wind_pred),
            'windgusts_10m_max': float(gust_pred),
            'precipitation_hours': float(prec_hours_pred)
        })

        for base_col in ['rain_sum','windspeed_10m_max','windgusts_10m_max','precipitation_hours']:

            if f"{base_col}_lag1" in current.index:
                current[f"{base_col}_lag3"] = current.get(f"{base_col}_lag2", current.get(f"{base_col}_lag3", 0))
                current[f"{base_col}_lag2"] = current.get(f"{base_col}_lag1", current.get(f"{base_col}_lag2", 0))
                current[f"{base_col}_lag1"] = yhat[['rain_sum','windspeed_10m_max','windgusts_10m_max','precipitation_hours'][0:4].index(base_col)] if False else None

        for base_col in ['rain_sum','windspeed_10m_max','windgusts_10m_max','precipitation_hours']:

            if f"{base_col}_lag1" in current.index:
                current[f"{base_col}_lag3"] = current.get(f"{base_col}_lag2", current.get(f"{base_col}_lag3", 0))
                current[f"{base_col}_lag2"] = current.get(f"{base_col}_lag1", current.get(f"{base_col}_lag2", 0))

                if base_col == 'rain_sum':
                    current[f"{base_col}_lag1"] = rain_pred
                elif base_col == 'windspeed_10m_max':
                    current[f"{base_col}_lag1"] = wind_pred
                elif base_col == 'windgusts_10m_max':
                    current[f"{base_col}_lag1"] = gust_pred
                elif base_col == 'precipitation_hours':
                    current[f"{base_col}_lag1"] = prec_hours_pred

        if ('year' in current.index) and ('month' in current.index) and ('day' in current.index):
            try:

                d = pd.Timestamp(year=int(current['year']), month=int(current['month']), day=int(current['day']))
                d_next = d + pd.Timedelta(days=1)
                current['year'] = d_next.year
                current['month'] = d_next.month
                current['day'] = d_next.day
                current['dayofyear'] = d_next.dayofyear
                current['week'] = int(d_next.isocalendar()[1])
            except Exception:
                pass
    return pd.DataFrame(preds_list)

12. Premium prediction function

In [30]:
def premium_predict(city, timeframe='next-day', param='rain_sum', agg='max', save_figs=True):
    """
    Returns:
      - preds_df: DataFrame with horizon predictions
      - aggregated_value: max/min of requested parameter across horizon
      - risk_suggestion: 'Low'/'Moderate'/'High' based on highest-risk day
      - figure paths: list of generated PNG paths for graphs
    """
    if timeframe == 'next-day':
        horizon = 1
    elif timeframe == 'next-week':
        horizon = 7
    elif timeframe == 'next-month':
        horizon = 30
    else:
        raise ValueError("Invalid timeframe")

    city_rows = df_sorted[df_sorted['city'] == city].copy()
    if city_rows.empty:
        try:
            city_idx = le_city.transform([city])[0]
            city_rows = df_sorted[df_sorted['city'] == city].copy()
        except Exception:
            raise ValueError("City not found in dataset")

    last_row = city_rows.sort_values(['year','month','day']).iloc[-1]

    start_row = X[X['city'] == le_city.transform([last_row['city']])[0]].iloc[-1] if 'city' in X.columns else X.iloc[-1]

    if start_row.shape[0] != len(X.columns):

        start_row = pd.Series(index=X.columns, dtype=float)
        for col in X.columns:
            if col in last_row.index:
                start_row[col] = last_row[col]
            else:
                start_row[col] = last_row.get(col, 0)
    start_row = start_row.fillna(0)

    preds_df = iterative_forecast(start_row, horizon=horizon)

    if agg == 'max':
        aggregated_value = preds_df[param].max()
    else:
        aggregated_value = preds_df[param].min()

    preds_df['risk'] = preds_df.apply(lambda r: classify_risk_from_preds(r['rain_sum'], r['windspeed_10m_max'], r['windgusts_10m_max'], r['precipitation_hours']), axis=1)
    if 'High' in preds_df['risk'].values:
        risk_suggestion = 'High'
    elif 'Moderate' in preds_df['risk'].values:
        risk_suggestion = 'Moderate'
    else:
        risk_suggestion = 'Low'

    fig_paths = []
    if save_figs:
        plt.figure()
        for c in target_cols:
            plt.plot(preds_df.index+1, preds_df[c], marker='o', label=c)
        plt.xlabel("Day")
        plt.ylabel("Value")
        plt.title(f"Predicted {', '.join(target_cols)} for {city} ({timeframe})")
        plt.legend()
        p1 = f"figures/predicted_targets_{city}_{timeframe}.png"
        plt.tight_layout()
        plt.savefig(p1)
        plt.close()
        fig_paths.append(p1)

        plt.figure()
        sns.barplot(x=preds_df.index+1, y=preds_df[param])
        plt.xlabel("Day")
        plt.ylabel(param)
        plt.title(f"{param} across {timeframe} in {city}")
        p2 = f"figures/param_{param}_{city}_{timeframe}.png"
        plt.tight_layout()
        plt.savefig(p2)
        plt.close()
        fig_paths.append(p2)

        plt.figure()
        preds_df['risk'].value_counts().plot(kind='pie', autopct='%1.1f%%')
        plt.title("Risk distribution across horizon")
        p3 = f"figures/risk_dist_{city}_{timeframe}.png"
        plt.savefig(p3)
        plt.close()
        fig_paths.append(p3)

    return preds_df, float(aggregated_value), risk_suggestion, fig_paths


13. Example usage of premium_predict

In [31]:
example_city = df_sorted['city'].iloc[0]

if isinstance(example_city, (int, np.integer)):
    example_city_name = le_city.inverse_transform([example_city])[0]
else:
    example_city_name = example_city

preds_df, agg_value, risk, figs = premium_predict(example_city_name, timeframe='next-week', param='rain_sum', agg='max')
print("Aggregated value:", agg_value, "Risk:", risk)
print("Generated figures:", figs)
preds_df.head()


Aggregated value: 5.0 Risk: Moderate
Generated figures: ['figures/predicted_targets_Athurugiriya_next-week.png', 'figures/param_rain_sum_Athurugiriya_next-week.png', 'figures/risk_dist_Athurugiriya_next-week.png']


,day,rain_sum,windspeed_10m_max,windgusts_10m_max,precipitation_hours,risk
0,1,5.0,16.401333,37.828667,23.006667,Moderate
1,2,5.0,15.762667,37.900000,22.573333,Moderate
2,3,5.0,15.614000,37.106667,22.453333,Moderate
3,4,5.0,15.754000,36.068667,22.266667,Moderate
4,5,5.0,15.661333,34.950667,21.533333,Moderate


14. Save final artifacts for Flask backend

In [ ]:
joblib.dump(best_model, "model_climatrix.pkl")
joblib.dump(scaler, "scaler_climatrix.pkl")
joblib.dump(le_city, "city_encoder_climatrix.pkl")
print("Pickle files saved: model_climatrix.pkl, scaler_climatrix.pkl, city_encoder_climatrix.pkl")

Pickle files saved: model_climatrix.pkl, scaler_climatrix.pkl, city_encoder_climatrix.pkl
